In [1]:
from nemo.collections.asr.models import EncDecSpeakerLabelModel
from IPython.display import Audio, display
import numpy as np
import torch
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F
import math
import os
import requests
import boto3
import wave
import sys
import contextlib
import webrtcvad
import collections
from tqdm import tqdm
from tqdm.notebook import tqdm

[NeMo W 2023-06-20 11:29:38 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-06-20 11:29:43 nemo_logging:349] /Users/ajaybati/miniconda3/envs/sanas/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


In [2]:
def play_audio(waveform, sample_rate, channel=-1):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    if num_channels == 1:
        display(Audio(waveform[0], rate=sample_rate))
    elif num_channels == 2:
        if channel != -1:
            display(Audio((waveform[channel]), rate=sample_rate))
        else:
            display(Audio((waveform[0], waveform[1]), rate=sample_rate))
    else:
        raise ValueError("Waveform with more than 2 channels are not supported.")

In [3]:
speaker_encoder = EncDecSpeakerLabelModel.from_pretrained(model_name="speakerverification_speakernet")

speaker_encoder.eval()

def get_emb(wav_path):
    return speaker_encoder.get_embedding(wav_path)

[NeMo I 2023-06-20 11:29:55 cloud:58] Found existing object /Users/ajaybati/.cache/torch/NeMo/NeMo_1.19.0rc0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
[NeMo I 2023-06-20 11:29:55 cloud:64] Re-using file from: /Users/ajaybati/.cache/torch/NeMo/NeMo_1.19.0rc0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo
[NeMo I 2023-06-20 11:29:55 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-06-20 11:29:55 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2023-06-20 11:29:55 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size:

[NeMo I 2023-06-20 11:29:55 features:289] PADDING: 16
[NeMo I 2023-06-20 11:29:55 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /Users/ajaybati/.cache/torch/NeMo/NeMo_1.19.0rc0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.


In [4]:
client = boto3.client(
    's3',
    aws_access_key_id = 'ENTER ACCESS KEY',
    aws_secret_access_key = 'ENTER SECRET ACCESS KEY',
    region_name = 'REGION NAME'
)
    
# Creating the high level object oriented interface
resource = boto3.resource(
    's3',
    aws_access_key_id = 'ENTER ACCESS KEY',
    aws_secret_access_key = 'ENTER SECRET ACCESS KEY',
    region_name = 'REGION NAME'
)

In [ ]:
bucket_name = "bucket name"
bucket_name

In [6]:
resp = client.list_objects(Bucket=bucket_name, Prefix='prefix', Delimiter="/")
speakers = [r['Prefix'] for r in resp["CommonPrefixes"]]

In [7]:
class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2) #*2 probably for 2 bytes for 16 bits
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.

    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.

    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.

    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.

    Arguments:

    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).

    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False
    time_stamps = []

    voiced_frames = []
    tup = [None,None]
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        # sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                # sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                tup[0] = ring_buffer[0][0].timestamp
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                # sys.stdout.write(f'-{frame.timestamp + frame.duration}')
                tup[1] = frame.timestamp + frame.duration
                time_stamps.append(tup)
                tup = [None, None]
                triggered = False
                # yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    # if triggered:
        # sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    # sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    # if voiced_frames:
    #     yield b''.join([f.bytes for f in voiced_frames])
    return time_stamps

In [8]:
RESAMPLE_RATE = 16000
import numpy as np
def getAudioSegs(key):
    obj = resource.Object(bucket_name, key)
    
    waveform, sample_rate = torchaudio.load(obj.get()["Body"])
    waveform = F.resample(waveform, sample_rate, RESAMPLE_RATE)[1].reshape(1,-1)
    
    torchaudio.save("test.wav", waveform, RESAMPLE_RATE, encoding="PCM_S", bits_per_sample=16)
    
    spf = wave.open("test.wav", 'rb')
    data = spf.readframes(spf.getnframes())
    
    vad = webrtcvad.Vad(3)
    frames = frame_generator(30, data, 16000)
    frames = list(frames)
    
    segments = vad_collector(16000, 30, 300, vad, frames)
    

    segments = np.array(segments).reshape(-1,2)

    frame_segments = segments * RESAMPLE_RATE
    
    speaker_segs = frame_segments[np.where((segments[:,1]-segments[:,0]) > 2)[0]]

    os.remove("test.wav")
    return [waveform[:,int(speaker_segs[i][0]):int(speaker_segs[i][1])] for i in range(len(speaker_segs))]

In [15]:
all_speaker_segs = [] #torch.load('all_speaker_segs0-5.pkl')

In [ ]:
#already configured, just run this cell
import time
for sprefix in tqdm(range(len(speakers)), desc=" outer", position=0):
    if sprefix % 5 == 0 and sprefix != 0:
        torch.save(all_speaker_segs[sprefix-5:sprefix], f'all_speaker_segs{sprefix-5}-{sprefix}.pkl')
        time.sleep(10)
    speaker_segs = []
    files = client.list_objects(Bucket=bucket_name, Prefix=speakers[sprefix])
    for audio_file in tqdm(range(len(files["Contents"])), desc=" inner loop", position=1, leave=True):
        try:
            key = files["Contents"][audio_file]["Key"]
            speaker_segs.append(getAudioSegs(key))
        except Exception as e:
            pass
    all_speaker_segs.append(speaker_segs)

In [34]:
torch.save(all_speaker_segs, f'all_speaker_segs2.pkl')

In [ ]:
# import torch
# torch.set_num_threads(1)

# model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')
# (get_speech_timestamps, _, read_audio, _, _) = utils

# speech_timestamps = get_speech_timestamps(waveform.flatten(), model, sampling_rate=16000, window_size_samples=1536, visualize_probs=True, return_seconds=True)

In [84]:
all_speaker_segs05 = torch.load('all_speaker_segs0-5.pkl')

In [85]:
speaker = all_speaker_segs05[0] #choose speaker
audio_file = speaker[0] #choose audio file
seg = audio_file[0] #choose segment from audio file
play_audio(seg, 16000)

In [80]:
speakerCount = 0
for x in range(0,55,5):
    all_speaker_segs05 = torch.load(f'all_speaker_segs{x}-{x+5}.pkl')
    print(f"Doing all_speaker_segs{x}-{x+5}.pkl, speakerCount: {speakerCount}")
    for spnum, speaker in enumerate(all_speaker_segs05):
        path = f"speakerWav/{speakerCount}"
        if not os.path.exists(path):
            os.mkdir(path)
        for audnum, audio_file in enumerate(speaker):
            if len(audio_file) != 0:
                for segnum, seg in enumerate(audio_file):
                    torchaudio.save(path+f"/{audnum}-{segnum}.wav", seg, 16000)
        speakerCount+=1

Doing all_speaker_segs0-5.pkl, speakerCount: 0
Doing all_speaker_segs5-10.pkl, speakerCount: 5
Doing all_speaker_segs10-15.pkl, speakerCount: 10
Doing all_speaker_segs15-20.pkl, speakerCount: 15
Doing all_speaker_segs20-25.pkl, speakerCount: 20
Doing all_speaker_segs25-30.pkl, speakerCount: 25
Doing all_speaker_segs30-35.pkl, speakerCount: 30
Doing all_speaker_segs35-40.pkl, speakerCount: 35
Doing all_speaker_segs40-45.pkl, speakerCount: 40
Doing all_speaker_segs45-50.pkl, speakerCount: 45
Doing all_speaker_segs50-55.pkl, speakerCount: 50


In [82]:
wv, sr = torchaudio.load("/Users/ajaybati/Documents/speakerNet/SpeakerNetTest/embeddings/speakerWav/0/0-0.wav")

In [10]:
dirs = sorted(os.listdir("speakerWav/"))[2:]

In [ ]:
speaker_encoder = EncDecSpeakerLabelModel.from_pretrained(model_name="speakerverification_speakernet")
ecapa = EncDecSpeakerLabelModel.from_pretrained(model_name='ecapa_tdnn')

ecapa.eval()
speaker_encoder.eval()

def get_emb(model, wav_path):
    return model.get_embedding(wav_path)

In [11]:
dirs[17:20]

['24', '25', '26']

In [ ]:
speakerEmbs = []
for dir in tqdm(range(len(dirs))):
    sp = []
    files = os.listdir(f"speakerWav/{int(dirs[dir])}/")
    for file in tqdm(range(len(files))):
        sp.append(get_emb(ecapa, f"speakerWav/{dirs[dir]}/{files[file]}"))
    speakerEmbs.append(sp)

In [127]:
# first = torch.load("speakerEmbs0-18.pkl")
# second = torch.load("speakerEmbs18-end.pkl")
# all = [f for f in first+second if f != []]

In [200]:
allT = [torch.stack(f).squeeze() for f in speakerEmbs if f != []]
torch.save(allT, "EcapaEmbs.pkl")

In [12]:
allTsp = torch.load("SpeakerNetEmbs.pkl")

In [13]:
allTec = torch.load("EcapaEmbs.pkl")

In [ ]:
allT_norms = [torch.linalg.norm(a,dim=1).reshape(1,-1) for a in allT]
allT_normx = [a.T @ a for a in allT_norms]

allT_dot = [torch.matmul(a,a.T) for a in allT]

cosSim2 = np.array([torch.triu(allT_dot[i]/allT_normx[i], diagonal=1)[torch.triu(allT_dot[i]/allT_normx[i], diagonal=1)!=0].median().item() for i in range(len(allT))])


In [304]:
cosSim

array([0.66796982, 0.62162811, 0.70326322, 0.60251921, 0.59233379,
       0.6242497 , 0.64582849, 0.68528628, 0.62247699, 0.56180799,
       0.63784027, 0.59009486, 0.60365552, 0.60692221, 0.75205094,
       0.59575427, 0.57736886, 0.65621805, 0.71825463, 0.64020061,
       0.60025859, 0.64164895, 0.52262169, 0.7402584 , 0.58957297,
       0.55084014, 0.59943479, 0.77541786, 0.66098481, 0.60084438,
       0.61643583, 0.60914886, 0.67998636, 0.65752673, 0.76466614,
       0.65778017, 0.57730323, 0.53992593, 0.60848325, 0.61246222,
       0.60465372, 0.5374077 , 0.67629731, 0.57500261, 0.54332018,
       0.55686426, 0.6351434 , 0.6086145 , 0.52175492, 0.68734747,
       0.50494456, 0.69177157, 0.60389757, 0.58709341])

In [309]:
cosSim2

array([0.66392142, 0.47538385, 0.64724976, 0.5005188 , 0.59452403,
       0.52057773, 0.53244597, 0.61026818, 0.4929412 , 0.37574393,
       0.53292829, 0.47691429, 0.51716417, 0.48866865, 0.61886287,
       0.42661509, 0.4757435 , 0.6099785 , 0.66809052, 0.56229812,
       0.58474183, 0.6438452 , 0.3057147 , 0.71264768, 0.36446193,
       0.43111449, 0.48512238, 0.67169023, 0.61145777, 0.55633891,
       0.47777063, 0.53788286, 0.62919575, 0.64494562, 0.63821673,
       0.62366474, 0.55708027, 0.44606593, 0.40453416, 0.52447599,
       0.47545204, 0.25464427, 0.60258102, 0.47670317, 0.29096904,
       0.42197186, 0.56326556, 0.57478118, 0.316594  , 0.61406595,
       0.1772697 , 0.68841738, 0.44018418, 0.34592167])

In [310]:
cosSim2 > cosSim

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [34]:
allTecMeans = torch.stack([allTec[i].mean(dim=0) for i in range(len(allTec))])
allTspMeans = torch.stack([allTsp[i].mean(dim=0) for i in range(len(allTsp))])

In [42]:
def cosim(embs1,embs2):
    return np.dot(embs1, embs2)/(np.linalg.norm(embs1)*np.linalg.norm(embs2))

In [43]:
all_scores = []
for x in range(len(allTecMeans)):
    for y in range(x+1, len(allTecMeans)):
        all_scores.append(cosim(allTecMeans[x],allTecMeans[y]))

In [53]:
np.median(np.array(all_scores))

0.34244406

In [48]:
all_scores2 = []
for x in range(len(allTspMeans)):
    for y in range(x+1, len(allTspMeans)):
        all_scores2.append(cosim(allTspMeans[x],allTspMeans[y]))

In [54]:
np.median(np.array(all_scores2))

0.7091421